# Simulación del Examen (Core)

## Contexto

La tarea es limpiar y explorar los datos para una compañía de tecnología emergente que desarrolla aplicaciones móviles. La empresa quiere mejorar la experiencia del usuario y aumentar la retención de usuarios en sus aplicaciones. Han recolectado datos sobre el uso de sus aplicaciones y quieren entender mejor cómo los usuarios interactúan con sus productos.

## Requisitos:

### 1. Limpieza de Datos:

El primer paso en este proceso es usar Python para limpiar los datos y dejarlos listos para el análisis. Deben:

* Identificar y abordar cualquier valor duplicado.
* Identificar y abordar cualquier dato que falte en este conjunto de datos. Trátenlos de forma adecuada. Incluyan una breve descripción en el método que usan para tratar con los valores que faltan junto con una justificación para el uso de ese método.
* Asegurarse de que todas las columnas coincidan con los tipos de datos enumerados en el diccionario de datos.
* Identificar y abordar cualquier inconsistencia en los valores categóricos (ejemplo: android, Android, ANDROID).
* Identificar y abordar cualquier punto de datos inapropiados o inusuales (ejemplo: tiempo de uso de 10000 horas en una semana).

### 2. Exploración de Datos:

El siguiente paso es completar una exploración de los datos usando Python. Esto debe incluir:

* Dos tipos diferentes de visualizaciones exploratorias univariantes. Cada visualización debe incluir una breve interpretación dentro del archivo de código.
* Dos tipos diferentes de visualizaciones exploratorias multivariantes. Cada visualización debe incluir una breve interpretación dentro del archivo de código.

Diccionario de Datos:

* user_id: Identificación única del usuario.
* app_version: Versión de la aplicación usada.
* platform: Plataforma del dispositivo (Android, iOS).
* session_duration: Duración de la sesión en minutos.
* number_of_sessions: Número de sesiones en un día.
* country: País del usuario.
* user_feedback: Puntuación de la experiencia del usuario (1-5).

### Resolución

Primero que nada, generamos los datos usando el script especificado en la tarea:

In [3]:
import pandas as pd
import numpy as np
import random

# Configuramos la semilla para reproducibilidad
np.random.seed(42)

# Generamos los datos sintéticos
user_id = range(1, 301)
app_version = np.random.choice(['1.0', '1.1', '1.2', '1.3'], 300)
platform = np.random.choice(['Android', 'iOS'], 300)
session_duration = np.random.randint(1, 180, 300)
number_of_sessions = np.random.randint(1, 20, 300)
country = np.random.choice(['USA', 'Canada', 'Mexico', 'UK', 'Germany', 'France', 'Spain', 'Italy'], 300)
user_feedback = np.random.randint(1, 6, 300)

# Creamos el DataFrame
data = pd.DataFrame({
    'user_id': user_id,
    'app_version': app_version,
    'platform': platform,
    'session_duration': session_duration,
    'number_of_sessions': number_of_sessions,
    'country': country,
    'user_feedback': user_feedback
})

# Guardamos los datos en un archivo CSV
data.to_csv('../data/user_app_data.csv', index=False)
print("Datos generados y guardados en '../data/user_app_data.csv'")

# Eliminamos el data frame para evitar consumo de memoria innecesario.
del data

Datos generados y guardados en '../data/user_app_data.csv'


Luego, procedemos a cargar los datos del csv generado.

In [5]:
camino_dataset = "../data/user_app_data.csv"
df = pd.read_csv(camino_dataset)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_id             300 non-null    int64  
 1   app_version         300 non-null    float64
 2   platform            300 non-null    object 
 3   session_duration    300 non-null    int64  
 4   number_of_sessions  300 non-null    int64  
 5   country             300 non-null    object 
 6   user_feedback       300 non-null    int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 16.5+ KB


Visualizamos las primeras columnas para dar una mirada rápida a los datos.

In [6]:
df.head(10)

,user_id,app_version,platform,session_duration,number_of_sessions,country,user_feedback
0,1,1.2,Android,109,11,France,1
1,2,1.3,Android,116,17,UK,2
2,3,1.0,Android,94,13,France,5
3,4,1.2,iOS,47,1,Canada,4
4,5,1.2,iOS,99,2,Canada,3
5,6,1.3,Android,55,9,Germany,4
6,7,1.0,iOS,168,3,Mexico,2
7,8,1.0,iOS,52,1,Canada,4
8,9,1.2,iOS,144,16,USA,1
9,10,1.1,iOS,13,6,France,2


Ahora, se procede a detectar cualquier tipo de anormalidad en los datos.

Al ser un dataset generado con valores aleatorios, es poco probable que existan correlaciones, pero se asume que no se posee dicho conocimiento para el análisis.

Lo primero que se puede notar en el DataSet es que no se especifica a que corresponde session_duration. Se asume que este valor es un promedio de duración por sesión, ya que no hay ningun otro dato en la descripción recibida.

In [11]:
# Para mayor claridad, se renombra el campo session_duration a avg_session_duration_minutes,
# indicando ya en la columna la duración de la sesión.
df.rename(columns={"session_duration": "avg_session_duration_minutes"}, inplace=True)
df.head(10)

,user_id,app_version,platform,avg_session_duration_minutes,number_of_sessions,country,user_feedback
0,1,1.2,Android,109,11,France,1
1,2,1.3,Android,116,17,UK,2
2,3,1.0,Android,94,13,France,5
3,4,1.2,iOS,47,1,Canada,4
4,5,1.2,iOS,99,2,Canada,3
5,6,1.3,Android,55,9,Germany,4
6,7,1.0,iOS,168,3,Mexico,2
7,8,1.0,iOS,52,1,Canada,4
8,9,1.2,iOS,144,16,USA,1
9,10,1.1,iOS,13,6,France,2


In [12]:
# Luego, se puede calcular el promedio de uso diario, multiplicando avg_session_duration_minutes
# por number_of_session.
df["daily_app_usage_minutes"] = df["avg_session_duration_minutes"] * df["number_of_sessions"]
df.head(10)

,user_id,app_version,platform,avg_session_duration_minutes,number_of_sessions,country,user_feedback,daily_app_usage_minutes
0,1,1.2,Android,109,11,France,1,1199
1,2,1.3,Android,116,17,UK,2,1972
2,3,1.0,Android,94,13,France,5,1222
3,4,1.2,iOS,47,1,Canada,4,47
4,5,1.2,iOS,99,2,Canada,3,198
5,6,1.3,Android,55,9,Germany,4,495
6,7,1.0,iOS,168,3,Mexico,2,504
7,8,1.0,iOS,52,1,Canada,4,52
8,9,1.2,iOS,144,16,USA,1,2304
9,10,1.1,iOS,13,6,France,2,78
